# Buffer1

In [24]:
import pandas as pd

In [25]:
merged_data=pd.read_csv('../data/processed/well_pchip.csv')
gage_to_wells_df = pd.read_csv('../data/processed/wells_with_catchment_info.csv')

In [ ]:
# Clean and prepare gage-to-wells mapping data
import pandas as pd

# Convert column names to lowercase and clean up
gage_to_wells_df.columns = gage_to_wells_df.columns.str.lower()

# Drop unnecessary columns
columns_to_drop = ['name', 'river', 'state', 'aquifer_na']
gage_to_wells_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Rename coordinate columns for clarity
gage_to_wells_df.rename(columns={
    'latitude': 'gage_lat',
    'longitude': 'gage_lon',
    'lat_dec': 'well_lat',
    'long_dec': 'well_lon'
}, inplace=True)

print("Data cleaning completed")
print(f"Cleaned data shape: {gage_to_wells_df.shape}")
print(gage_to_wells_df.head())

In [ ]:
# Data merge and validation
print("=== Merging well data with gage information ===")

# Execute the merge
print("Executing merge...")
filtered_data = pd.merge(
    merged_data,  # Use the merged_data from the previous cell
    gage_to_wells_df[['gage_id', 'well_id', 'well_lat', 'well_lon']],
    on='well_id',
    how='inner'
)

print(f"Merged data shape: {filtered_data.shape}")
print(f"Unique gage_id in result: {sorted(filtered_data['gage_id'].unique())}")

# Handle duplicate columns from merge
if 'well_lat_x' in filtered_data.columns:
    filtered_data.drop(columns=['well_lat_x', 'well_lon_x'], inplace=True)

if 'well_lat_y' in filtered_data.columns:
    filtered_data.rename(columns={'well_lat_y': 'well_lat', 'well_lon_y': 'well_lon'}, inplace=True)

print(f"Final processed data shape: {filtered_data.shape}")
print(filtered_data.head())

In [29]:
gage_info = pd.read_csv('../data/raw/streamflow/gsl_nwm_gage.csv')
gage_info

,id,name,River,latitude,longitude,elevation_m,state
0,10011200,WEST FORK BEAR RIVER AT WHITNEY DAM - NR OAKLE...,WEST FORK BEAR RIVER,40.841614,-110.927119,2797,Utah
1,10011500,BEAR RIVER NEAR UTAH-WYOMING STATE LINE,BEAR RIVER,40.965225,-110.853508,2432,Utah
2,10015700,SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...,SULPHUR CREEK,41.129114,-110.806563,2205,Wyoming
3,10015900,SULPHUR CREEK BL RES. - NEAR EVANSTON - WYO.,SULPHUR CREEK,41.156058,-110.835176,2173,Wyoming
4,10016900,BEAR RIVER AT EVANSTON - WY,BEAR RIVER,41.270224,-110.963793,2057,Wyoming
...,...,...,...,...,...,...,...
73,10168500,BIG COTTONWOOD CR NR SALT LAKE CITY UTAH,LITTLE COTTONWOOD CREEK,40.618559,-111.781876,1527,Utah
74,10171000,JORDAN RIVER @ 1700 SOUTH @ SALT LAKE CITY - UT,JORDAN RIVER,40.733557,-111.923270,1294,Utah
75,10172700,VERNON CREEK NEAR VERNON - UT,VERNON CREEK,39.979391,-112.380230,1898,Utah
76,10172860,WARM CREEK NEAR GANDY - UT,WARM CREEK,39.459528,-114.023722,1573,Utah


In [30]:
# Merge gage_info with filtered_data to add gage coordinates
filtered_data = pd.merge(
    filtered_data,
    gage_info[['id', 'latitude', 'longitude']],
    left_on='gage_id',
    right_on='id',
    how='left'
)

# Rename the columns
filtered_data.rename(columns={
    'latitude': 'gage_lat',
    'longitude': 'gage_lon'
}, inplace=True)

# Drop the redundant id column
filtered_data.drop('id', axis=1, inplace=True)


In [31]:
filtered_data.to_csv('../data/processed/after_buffer1.csv', index=False)

In [32]:
# Calculate the number of unique wells
unique_wells_count = filtered_data['well_id'].nunique()

# Display the result
print(f"Number of unique wells: {unique_wells_count}")

unique_gages_count = filtered_data['gage_id'].nunique()
print(f"Number of unique gages: {unique_gages_count}")

Number of unique wells: 1141
Number of unique gages: 8


# buffer2

In [ ]:
import pandas as pd
import numpy as np

def filter_and_analyze_wte(filtered_data, merged_df, distance_buffer_meters=30, delta_bins=None):
    # Create copy to avoid SettingWithCopyWarning
    filtered_data = filtered_data.copy()
    
    # Convert WTE from feet to meters
    filtered_data['wte_meters'] = filtered_data['wte'] * 0.3048

    # Merge filtered_data with merged_df to get reach_elev
    merged_data = pd.merge(
        filtered_data,
        merged_df[['well_id', 'reach_elev_m']],
        on='well_id',
        how='inner'
    )

    # Calculate the difference between reach elevation and WTE
    merged_data['delta_elev'] = merged_data['reach_elev_m'] - merged_data['wte_meters']

    # Filter out values where the difference is greater than the distance buffer
    filtered_result = merged_data[merged_data['delta_elev'] <= distance_buffer_meters].copy()

    # Define default delta bins if not provided
    if delta_bins is None:
        delta_bins = [-float('inf'), -20, -10, -5, 0, 5, 10, 20, 30, 50, 75, 100, float('inf')]

    # Bin the delta_elev values
    bin_labels = [f"< {delta_bins[1]}"] + \
                 [f"{delta_bins[i]} to {delta_bins[i+1]}" for i in range(1, len(delta_bins)-2)] + \
                 [f">= {delta_bins[-2]}"]
    
    # Fix pandas warning
    filtered_result.loc[:, 'delta_bin'] = pd.cut(filtered_result['delta_elev'], bins=delta_bins, labels=bin_labels)

    # Calculate statistics - fix FutureWarning
    total_measurements = len(filtered_result)
    dist_stats = filtered_result.groupby('delta_bin', observed=True).size().reset_index(name='count')
    dist_stats['percentage'] = (dist_stats['count'] / total_measurements * 100).round(2)

    # Display statistics
    print(dist_stats)

    return filtered_result, dist_stats

# Efficient data loading - load only when needed
print("Loading data...")
filtered_data = pd.read_csv('../data/processed/after_buffer1.csv')
merged_df = pd.read_csv('../data/processed/well_reach.csv')

print(f"Data loading complete - filtered_data: {filtered_data.shape}, merged_df: {merged_df.shape}")

In [34]:
filtered_result, dist_stats = filter_and_analyze_wte(filtered_data, merged_df, distance_buffer_meters=30)

    delta_bin   count  percentage
0       < -20  653359       12.92
1  -20 to -10  477643        9.45
2   -10 to -5  512136       10.13
3     -5 to 0  638485       12.63
4      0 to 5  738653       14.61
5     5 to 10  784842       15.52
6    10 to 20  794499       15.71
7    20 to 30  456975        9.04


In [35]:
filtered_result

,well_id,date,wte,gse,gage_id,well_lat,well_lon,gage_lat,gage_lon,wte_meters,reach_elev_m,delta_elev,delta_bin
7178,394643111291401,1971-09-15,5953.300000,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.565840,1808.5,-6.065840,-10 to -5
7179,394643111291401,1971-09-16,5953.312390,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.569616,1808.5,-6.069616,-10 to -5
7180,394643111291401,1971-09-17,5953.324794,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.573397,1808.5,-6.073397,-10 to -5
7181,394643111291401,1971-09-18,5953.337211,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.577182,1808.5,-6.077182,-10 to -5
7182,394643111291401,1971-09-19,5953.349641,5955.0,10152000.0,39.778571,-111.487970,40.150232,-111.726039,1814.580971,1808.5,-6.080971,-10 to -5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6884353,424553111364801,1968-01-12,5998.253943,6097.0,10126000.0,42.764644,-111.614119,41.576321,-112.100782,1828.267802,1835.5,7.232198,5 to 10
6884354,424553111364801,1968-01-13,5998.286603,6097.0,10126000.0,42.764644,-111.614119,41.576321,-112.100782,1828.277757,1835.5,7.222243,5 to 10
6884355,424553111364801,1968-01-14,5998.320158,6097.0,10126000.0,42.764644,-111.614119,41.576321,-112.100782,1828.287984,1835.5,7.212016,5 to 10
6884356,424553111364801,1968-01-15,5998.354619,6097.0,10126000.0,42.764644,-111.614119,41.576321,-112.100782,1828.298488,1835.5,7.201512,5 to 10


In [36]:
filtered_result.to_csv('../data/processed/after_buffer2_30m.csv', index=False)

## use q dates to filter after buffer2

In [37]:
filtered_data.head()

,well_id,date,wte,gse,gage_id,well_lat,well_lon,gage_lat,gage_lon
0,394618111291501,1970-03-14,5963.000000,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039
1,394618111291501,1970-03-15,5963.000764,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039
2,394618111291501,1970-03-16,5963.001528,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039
3,394618111291501,1970-03-17,5963.002291,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039
4,394618111291501,1970-03-18,5963.003054,5980.0,10152000.0,39.771627,-111.488248,40.150232,-111.726039


In [39]:
filtered_result.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6894716 entries, 0 to 6894715
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   well_id   int64  
 1   date      object 
 2   wte       float64
 3   gse       float64
 4   gage_id   float64
 5   well_lat  float64
 6   well_lon  float64
 7   gage_lat  float64
 8   gage_lon  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 473.4+ MB


In [ ]:
# Load streamflow data
compiled_data = pd.read_csv('../data/processed/streamflow/q_bfd_1.csv')
print(f"Streamflow data loaded: {compiled_data.shape}")
print(compiled_data.head())

In [ ]:
# Merge with streamflow data
print("=== Merging with streamflow data ===")

# Standardize data types for merge
print("Standardizing data types...")
filtered_result['gage_id'] = filtered_result['gage_id'].astype('int64')
compiled_data['gage_id'] = compiled_data['gage_id'].astype('int64')

# Check overlapping gage IDs
filtered_gages = set(filtered_result['gage_id'])
compiled_gages = set(compiled_data['gage_id'])
overlap = filtered_gages.intersection(compiled_gages)

print(f"Overlapping gage_id count: {len(overlap)}")
print(f"Overlapping gage_ids: {sorted(list(overlap))}")

if len(overlap) > 0:
    print(f"Date ranges:")
    print(f"  filtered_result: {filtered_result['date'].min()} - {filtered_result['date'].max()}")
    print(f"  compiled_data: {compiled_data['date'].min()} - {compiled_data['date'].max()}")
    
    # Create wte_meters column if it doesn't exist
    if 'wte_meters' not in filtered_result.columns:
        print("Creating wte_meters column...")
        filtered_result['wte_meters'] = filtered_result['wte'] * 0.3048
    
    # Perform efficient merge
    print("Starting merge...")
    available_cols = ['well_id', 'date', 'wte', 'gse', 'gage_id']
    for col in ['well_lat', 'well_lon', 'gage_lat', 'gage_lon', 'wte_meters']:
        if col in filtered_result.columns:
            available_cols.append(col)
    
    q_buffer2_pair = pd.merge(
        filtered_result[available_cols],
        compiled_data[['gage_id', 'date', 'q', 'bfd']],
        on=['gage_id', 'date'],
        how='inner'
    )
    
    print(f"Merge successful! Records: {len(q_buffer2_pair):,}")
    
    if len(q_buffer2_pair) > 0:
        print(f"Final data year range: {pd.to_datetime(q_buffer2_pair['date']).dt.year.min()} - {pd.to_datetime(q_buffer2_pair['date']).dt.year.max()}")
        print(f"Included gage_ids: {sorted(q_buffer2_pair['gage_id'].unique())}")
        print("\nFirst 5 rows sample:")
        print(q_buffer2_pair[['well_id', 'date', 'wte', 'gage_id', 'q']].head())
    else:
        print("No data after merge")
        q_buffer2_pair = pd.DataFrame()
else:
    print("No overlapping gage_ids found")
    q_buffer2_pair = pd.DataFrame()

print("Processing complete!")

In [ ]:
# Quick data summary check
print("=== Data Summary ===")
print(f"filtered_result: {len(filtered_result):,} rows")
print(f"compiled_data: {len(compiled_data):,} rows")
print(f"Unique gage IDs - filtered: {len(filtered_result['gage_id'].unique())}, compiled: {len(compiled_data['gage_id'].unique())}")

# Check overlap
overlap = set(filtered_result['gage_id']).intersection(set(compiled_data['gage_id']))
print(f"Overlapping gage_ids: {len(overlap)} - {sorted(list(overlap))}")

print(f"Date ranges - filtered: {filtered_result['date'].min()} to {filtered_result['date'].max()}")
print(f"Date ranges - compiled: {compiled_data['date'].min()} to {compiled_data['date'].max()}")

In [45]:
q_buffer2_pair.to_csv('../data/processed/q_buffer2_pair_test1.csv', index=False)